<a href="https://colab.research.google.com/github/jcandane/Lie_Algebras/blob/main/lie_algebra_su.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adjoint Representation of $\mathfrak{su}(n)$ Lie Algebras

This consisnts of 3 parts, 


1.   Off-diagonal 1 $\left( \frac{n(n-1)}{2} \text{ elements}\right)$,
2.   off-diagonal $i$ $\left(\frac{n(n-1)}{2} \text{ elements} \right)$,
3.   on-diagonal part $\left( n-1 \text{ elements}\right)$.

i.e. $\frac{n(n-1)}{2} + \frac{n(n-1)}{2} + n -1 = n^2 -1$ number of elements in the basis of the $\mathfrak{su}(n)$ Lie-Algebra

In [2]:
import numpy as np

N = 4

## diagonal part
A=np.tril( np.ones((N-1,N)) )
i,j = np.indices(A.shape)
A[i==j-1] = -np.arange(1,N)

ZZ = np.zeros((N-1, N, N))
for i in range(len(A)):
    ZZ[i] = np.diag(A[i])

## off diagonal part 1
a, b = np.where( (np.ones((N,N)) - np.eye(N)) == 1. )
i = a[a < b]
j = b[a < b]

Zr = np.zeros((N*(N-1)//2, N, N), dtype=np.complex128)
Zi = np.zeros((N*(N-1)//2, N, N), dtype=np.complex128)
Zr[np.arange(len(i)), i, j] = 1
Zr[np.arange(len(i)), j, i] = 1
Zi[np.arange(len(i)), i, j] = -1j
Zi[np.arange(len(i)), j, i] =  1j

lie_algebra = np.concatenate( (ZZ,np.concatenate((Zr, Zi))))

### http://www.ejtp.com/articles/ejtpv10i28p9.pdf

def get_su(N):

    A=np.tril( np.ones((N-1,N)) )
    i,j = np.indices(A.shape)
    A[i==j-1] = -np.arange(1,N)

    ZZ = np.zeros((N-1, N, N), dtype=np.complex128)
    for i in range(len(A)):
        ZZ[i] = np.diag(A[i]) / np.sqrt( (i+1)*(i+2)/2 )

    ## off diagonal part 1
    a, b = np.where( (np.ones((N,N)) - np.eye(N)) == 1. )
    i = a[a < b]
    j = b[a < b]

    Zr = np.zeros((N*(N-1)//2, N, N), dtype=np.complex128)
    Zi = np.zeros((N*(N-1)//2, N, N), dtype=np.complex128)
    Zr[np.arange(len(i)), i, j] = 1
    Zr[np.arange(len(i)), j, i] = 1
    Zi[np.arange(len(i)), i, j] = -1j
    Zi[np.arange(len(i)), j, i] =  1j

    return np.concatenate( (ZZ,np.concatenate((Zr, Zi))))

This is normalized to: $\sigma^{Ipq}\sigma^{Jpq} = 2\delta^{IJ}$

In [3]:
λ = get_su(12)
print( np.allclose( np.diag( np.einsum("Ipq, Jqp -> IJ", λ, λ).real ), 2*np.ones(λ.shape[0]) ) )

True


## Now lets compute the Structure Constants, $f^{IJK}$


$\begin{aligned}
f^{IJK}&= -\frac{i}{4} \left( \, \lambda^I_{pq}[\lambda,\lambda]^{JK}_{pq} \,\right)^{IJK}
\end{aligned}$

In [4]:
def get_struture_constants(λ):
  return -1j/4 * np.einsum("Ipq, JKpq -> IJK", λ, (np.einsum("Ipr, Jrq -> IJpq", λ, λ) - np.einsum("Jpr, Irq -> IJpq", λ, λ)))

## Test for SU(2)

In [5]:
λ = get_su(2)
f_IJK = get_struture_constants(λ)

#λ[2] @ λ[3] - λ[3] @ λ[2]

print(-1j/2 *(λ[0] @ λ[1] - λ[1] @ λ[0]) == np.einsum("IJK, Kpq -> IJpq", f_IJK, λ)[0,1])
print(-1j/2 *(λ[1] @ λ[2] - λ[2] @ λ[1]) == np.einsum("IJK, Kpq -> IJpq", f_IJK, λ)[1,2])
print(-1j/2 *(λ[1] @ λ[0] - λ[0] @ λ[1]) == np.einsum("IJK, Kpq -> IJpq", f_IJK, λ)[1,0])

[[ True  True]
 [ True  True]]
[[ True  True]
 [ True  True]]
[[ True  True]
 [ True  True]]


## Test for SU(3)

In [6]:
λ = get_su(3)
np.allclose( np.diag( np.einsum("Ipq, Jqp -> IJ", λ, λ).real ), 2*np.ones(λ.shape[0]) )

λλ    = -1j/4 * (np.einsum("Ipr, Jrq -> IJpq", λ, λ) - np.einsum("Jpr, Irq -> IJpq", λ, λ))
f_IJK = np.einsum("Ipq, JKpq -> IJK", λ, λλ)

## λ[3], λ[7], λ[1] nonzero Gellman Matrices
print( -1j/2 *(λ[3] @ λ[7] - λ[7] @ λ[3]) == np.einsum("IJK, Kpq -> IJpq", f_IJK, λ)[3,7] )

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


## Jacobi Identity

$ [X,[Y,Z]] + [Y,[Z,X]] + [Z,[X,Y]] = 0.$

$\left[ λ^I_{pr} \left[ λ^J, λ^K \right]_{rq} \right]^{IJK}_{pq} + \left[λ^J_{pr}, \left[ λ^K, λ^I \right]_{rq} \right]^{JKI}_{pq} + \left[ λ^K_{pr} \left[ λ^I, λ^J \right]_{rq}\right]^{KIJ}_{pq} = 0 $

In [27]:
λλ  = (np.einsum("Ipr, Jrq -> IJpq", λ, λ) - np.einsum("Jpr, Irq -> IJpq", λ, λ))
λλλ =  np.einsum("IJpr, Krq -> IJKpq", λλ, λ) - np.einsum("Ipr, JKrq -> IJKpq", λ, λλ)

##λ[3], λ[7] @ λ[1]
def commu(A, B, C):
    BC = B @ C - C @ B
    return A @ BC - BC @ A

#λ[3] @ (λ[7] @ λ[1] - λ[1] @ λ[7]) - (λ[7] @ λ[1] - λ[1] @ λ[7]) @ λ[3]
commu(λ[3], λ[7], λ[1]) + commu(λ[7], λ[1], λ[3]) + commu(λ[1], λ[3], λ[7])

np.allclose( λλλ + λλλ.swapaxes(0,1).swapaxes(0,2) + λλλ.swapaxes(0,1).swapaxes(0,2).swapaxes(0,1).swapaxes(0,2), np.zeros(λλλ.shape) )

True